In [7]:
!pwd

/Users/tazoe/tazoWork/vis_sumo/notebook


In [1]:
import sys, os
import re
from pickle import load, loads
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd
import numpy as np
from collections import Counter
output_notebook()

Loading BokehJS ...

## Load Data
- 各取り組みの結果は **[力士名(東), 力士名(西), 勝敗(Trueで東の勝利), 決まり手]** の順に格納
- **`basyo_dict`** : 取り組み結果をまとめたDataFrame． 年と月をキーに持つ．

In [10]:
data_dir = Path('../data/')
basyo_dir = [*(data_dir/'2018').iterdir()] #, *(data_dir/'2019').iterdir(), data_dir/'2020'/'01']
## 日にち抽出
pattern = '.*day(\d+)'
repatter = re.compile(pattern)

basyo_dict = {2018: {}, 2019: {}, 2020: {}}  
columns = ['day', 'east', 'west', 'which_win', 'kimarite']
for basyo in basyo_dir:
    y, m = int(basyo.parent.name), int(basyo.name)
    basyo_df = pd.DataFrame(columns=columns)
    for torikumis in basyo.iterdir():
        stem = str(torikumis.stem)
        result = repatter.match(stem)
        day = int(result.group(1))        
        with torikumis.open('rb') as f:
            results = pd.DataFrame(load(f), columns=['east', 'west', 'which_win', 'kimarite'])
        results['day'] = day
        results = results.loc[:, columns]
        basyo_df = pd.concat([basyo_df, results])
    basyo_df = basyo_df.sort_values('day')
    basyo_df = basyo_df.reset_index(drop=True)
    basyo_dict[y][m] = basyo_df

print(basyo_dict.keys(), basyo_dict[2019].keys())

dict_keys([2018, 2019, 2020]) dict_keys([])


In [11]:
basyo_dict[2018][1]

,day,east,west,which_win,kimarite
0,1,宝富士,遠藤,True,掬い投げ
1,1,竜電,錦木,False,上手投げ
2,1,石浦,豊山,False,寄り倒し
3,1,阿炎,大栄翔,True,突き落とし
4,1,豪風,輝,True,押し出し
5,1,蒼国来,大翔丸,True,押し出し
6,1,琴勇輝,安美錦,False,叩き込み
7,1,照ノ富士,千代丸,True,寄り切り
8,1,松鳳山,魁聖,True,小手投げ
9,1,栃煌山,千代の国,False,押し出し


### Bokehで折れ線グラフを試す．

In [12]:
year, month = 2018, 1
data = basyo_dict[year][month]
rikishis = list(set(list(data['east']) +list(data['west'])))
rikishi_syori = np.zeros((len(rikishis), 15), dtype=int)
# rikishi_counter = Counter(dict(zip(rikishis, np.zeros(len(rikishis), dtype=int))))
rikishi_index = dict(zip(rikishis, np.arange(len(rikishis))))
print(rikishi_index)

for day in np.arange(15):
    d = data[data.day==day+1]
    win_east = (d[d.which_win==True])['east']
    win_west = (d[d.which_win==False])['west']
    winners = pd.concat([win_east, win_west])
    rikishi_syori[:, day] += rikishi_syori[:, day-1]
    for rikishi in winners:
        rikishi_syori[rikishi_index[rikishi]][day] += 1

print(rikishi_syori)

{'大翔丸': 0, '御嶽海': 1, '錦木': 2, '石浦': 3, '千代丸': 4, '大栄翔': 5, '北勝富士': 6, '荒鷲': 7, '照ノ富士': 8, '千代翔馬': 9, '遠藤': 10, '魁聖': 11, '輝': 12, '玉鷲': 13, '隠岐の海': 14, '正代': 15, '碧山': 16, '竜電': 17, '琴勇輝': 18, '宝富士': 19, '松鳳山': 20, '阿炎': 21, '鶴竜': 22, '勢': 23, '稀勢の里': 24, '安美錦': 25, '妙義龍': 26, '大奄美': 27, '嘉風': 28, '栃煌山': 29, '豊山': 30, '英乃海': 31, '豪栄道': 32, '旭大星': 33, '豪風': 34, '阿武咲': 35, '貴景勝': 36, '千代大龍': 37, '琴奨菊': 38, '朝乃山': 39, '千代の国': 40, '栃ノ心': 41, '高安': 42, '東龍': 43, '蒼国来': 44, '逸ノ城': 45, '白鵬': 46}
[[ 0  1  2  3  4  4  5  5  6  6  6  7  8  9  9]
 [ 0  0  0  0  0  0  0  1  2  3  4  5  5  6  7]
 [ 1  2  2  2  3  3  4  5  5  5  6  6  7  7  7]
 [ 0  0  0  1  2  3  3  4  4  5  5  6  6  6  6]
 [ 0  1  1  1  2  2  2  3  4  4  4  5  5  6  6]
 [ 0  1  1  1  1  1  1  1  2  2  3  4  5  5  6]
 [ 1  2  2  3  4  5  5  6  7  8  9  9  9 10 11]
 [ 0  1  1  2  3  3  3  4  5  5  5  6  7  7  7]
 [ 1  2  2  2  2  2  2  2  2  2  3  4  5  6  7]
 [ 0  0  0  1  1  2  3  3  4  4  5  6  7  8  9]
 [ 0  0  1  1  1  2  2  3 

In [265]:
p = figure(title=f'{year}年初場所', x_axis_label='日にち', y_axis_label='勝利回数')
p.title.text = f'{year}年初場所'
強い力士 = rikishi_syori[np.argsort(rikishi_syori[:, 14])][::-1]
num = 12
X = np.arange(1, 16)
for num_win in 強い力士[:num]:
    p.line(X, num_win)
    p.circle(X, num_win)
show(p)


In [1]:
import sys, os
import numpy as np
import igraph
from igraph import Graph, summary
import networkx as nx
sys.path.append('/Users/tazoe/Dropbox/research/MultiplexNetworkVis/multiplex_svf/data/')
from aarhus import Aarhus

def igraph2nx(g_i: Graph) -> nx.Graph:
    """
    igraph.Graphをnx.Graphに変更するための関数
    """
    edges = [list(e) for e in g_i.get_edgelist()]
    G = nx.Graph(edges)
    return G

In [16]:
import sys, os
import numpy as np
from igraph import Graph, summary
from bokeh.io import output_notebook, show
from bokeh.models import Plot, Range1d, GraphRenderer, Oval, StaticLayoutProvider, Circle, MultiLine, \
                        HoverTool, TapTool, BoxSelectTool
from bokeh.models.widgets import Select, Button
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.layouts import Column
from bokeh.palettes import Spectral4
from bokeh.events import Tap, MouseMove, Press, PressUp
sys.path.append('/Users/tazoe/Dropbox/research/MultiplexNetworkVis/multiplex_svf/data/')
from aarhus import Aarhus
from copenhagen_network import Copenhagen
output_notebook()

def change_layer(p: Plot, graph: Graph, ratio:int=110):
    """
    描画するグラフを変更するための処理
    """
    p.renderers.clear()
    graph_renderer = GraphRenderer()
    node_indices = range(len(graph.vs))
    edges = np.array(graph.get_edgelist())
    start, end = edges[:, 0], edges[:, 1]

    graph_renderer.node_renderer.data_source.add(node_indices, 'index')
    graph_renderer.node_renderer.glyph = Circle(size=12, fill_color=Spectral4[0])
    graph_renderer.node_renderer.selection_glyph = Circle(size=40, fill_color=Spectral4[2])
    graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

    graph_renderer.edge_renderer.data_source.data = dict(start=start, end=end)
    graph_renderer.edge_renderer.glyph = MultiLine()
    graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2])
    graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

    layout = graph.layout_fruchterman_reingold()
    centroid = layout.centroid()
    layout = np.array(layout) - np.array(centroid)
    abs_max = np.max(np.abs(np.array(graph.layout_fruchterman_reingold()).flatten()))
    layout = list(layout* ratio / abs_max)
    graph_layout = dict(zip(node_indices, layout))
    graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
    graph_renderer.selection_policy = NodesAndLinkedEdges()
    graph_renderer.inspection_policy = EdgesAndLinkedNodes()
    p.renderers.append(graph_renderer)
    return np.array(layout)
    
layers, layer_dict = Aarhus()#Copenhagen()
def set_layer(attr, old_layer_name, new_layer_name):
    global p, layers, layer_dict
    layer_names = list(layer_dict.values())
    index = layer_names.index(new_layer_name)
    change_layer(p, layers[index])
    
# Widgetの作成
down_button = Button(label='Down Pos', button_type='success')
select = Select(title='Layer Name', value='lunch', options=list(layer_dict.values()))
p = Plot(plot_width=400, plot_height=400, x_range=Range1d(-110, 110), y_range=Range1d(-110,110))
p.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())
p.title.text = 'Graph Drawing'

# 描画処理
layer = layers[0]
layout = change_layer(p, layer)

# 描画変更
select.on_change('value', set_layer)

def down_pos():
    global p, layout
    a = p.renderers.pop()
    layout_provider = a.layout_provider
    node_indices = list(layout_provider.graph_layout.keys())
    layout_dict = map(lambda v: [v[0], v[1]-10], layout_provider.graph_layout.values())
    graph_layout = dict(zip(node_indices, layout_dict))
    layout_provider.graph_layout=graph_layout
    p.renderers.append(a)
down_button.on_click(down_pos)

def callback_tap(event):
    print(dir(event))
    print(event.event_name)

def callback_move(event):
    print('move called')
    print(event.sx, event.sy, event.x, event.y)
    
p.on_event(Tap, callback_tap)
p.on_event(PressUp, callback_move)

# レイアウト
RegionLevels = Column(select, p, down_button)
def Region(doc):
    doc.add_root(RegionLevels)
show(Region)

ImportError: cannot import name 'PressTool'